# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3 as sq
import re
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin

In [4]:
# load data from database
engine = create_engine('sqlite:///Disaster_Messages.db')
df = pd.read_sql("SELECT * FROM messages_and_categories", sq.connect("/content/drive/MyDrive/datasets/ds-data-engineering/Disaster_Messages.db"))

#testing purpose
df = df[0:5000]

X = df["message"]
Y = df.drop(["message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    '''
    INPUT:
    text - raw string

    OUTPUT
    tokens - list of token word
    '''
    # make all lowercase and delete punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    #tokenize and lemmatize
    tokens = word_tokenize(text.lower())
    tokens = [PorterStemmer().stem(token) for token in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_preds = pipeline.predict(x_test)
y_preds = pd.DataFrame(y_preds, columns=y_test.columns)

In [9]:
for col_name in y_preds.columns[0:3]:
  print(classification_report(y_test[col_name], y_preds[col_name]))

              precision    recall  f1-score   support

           0       0.65      0.34      0.45       273
           1       0.83      0.95      0.89       969
           2       0.00      0.00      0.00         8

    accuracy                           0.81      1250
   macro avg       0.49      0.43      0.44      1250
weighted avg       0.79      0.81      0.78      1250

              precision    recall  f1-score   support

           0       0.82      0.82      0.82       669
           1       0.79      0.79      0.79       581

    accuracy                           0.80      1250
   macro avg       0.80      0.80      0.80      1250
weighted avg       0.80      0.80      0.80      1250

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1248
           1       0.00      0.00      0.00         2

    accuracy                           1.00      1250
   macro avg       0.50      0.50      0.50      1250
weighted avg       1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'clf__estimator__n_estimators': [50,100,200,400],
    'clf__estimator__min_samples_split': [2, 3, 5],
    'tfidf__use_idf': [True, False]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(x_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_preds = cv.predict(x_test)
y_preds = pd.DataFrame(y_preds, columns=y_test.columns)

In [ ]:
for col_name in y_preds.columns[0:3]:
  print(classification_report(y_test[col_name], y_preds[col_name]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

  def starting_verb(self, text):
    sentence_list = nltk.sent_tokenize(text)
    for sentence in sentence_list:
      pos_tags = nltk.pos_tag(tokenize(sentence))
      if len(pos_tags) != 0:
        first_word, first_tag = pos_tags[0]
      else:
        first_word, first_tag = (' ', ' ')
      if first_tag in ['VB', 'VBP'] or first_word == 'RT':
        return True
    return False

  def fit(self, x, y=None):
    return self

  def transform(self, X):
    X_tagged = pd.Series(X).apply(self.starting_verb)
    return pd.DataFrame(X_tagged)

In [ ]:
pipeline2 = Pipeline([
  ('feature_union', FeatureUnion([
    ('text_pipeline', Pipeline([
      ('vect', CountVectorizer(tokenizer=tokenize)),
      ('tfidf', TfidfTransformer())
    ])),
    ('verb_extractor', StartingVerbExtractor())                   
  ])),
  ('clf', MultiOutputClassifier(LogisticRegression()))
])

In [ ]:
for col_name in Y.columns:
  if Y[col_name].sum() == 0:
    print(col_name)

In [ ]:
pipeline2.fit(x_train, y_train.drop(columns=['child_alone']))

In [ ]:
y_preds2 = pd.DataFrame(pipeline2.predict(x_test), columns=y_test.drop(columns=['child_alone']).columns)

In [ ]:
for col_name in y_preds.columns[0:3]:
  print(classification_report(y_test[col_name], y_preds2[col_name]))

### 9. Export your model as a pickle file

In [ ]:
with open('/content/drive/MyDrive/datasets/ds-data-engineering/model.pickle', 'wb') as dir:
    pickle.dump(pipeline2, dir)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.